# init

In [1]:
import pygame
import numpy as np
import numpy.linalg as lin
import sys
import importlib
import itertools
import time

import Clipping

import Geometry
from Geometry import Point
from camera import Camera
import draw
import vec
from inputs import input_update

# reload modules

In [16]:

importlib.reload(sys.modules['Clipping'])
importlib.reload(sys.modules['Geometry'])
importlib.reload(sys.modules['vec'])
importlib.reload(sys.modules['util'])
importlib.reload(sys.modules['opengl_draw_2d'])
importlib.reload(sys.modules['opengl_draw_3d'])
import Geometry

importlib.reload(sys.modules['camera'])
from camera import Camera
importlib.reload(sys.modules['inputs'])
from inputs import input_update

importlib.reload(sys.modules['draw'])
import draw



# build shapes

In [3]:
from colors import *
    
d = 4 #set dimensionality
cube_face_colors = [RED,YELLOW,GREEN,CYAN,MAGENTA, WHITE]
tess_face_colors = cube_face_colors + [ORANGE,PURPLE]

Geometry.Face.n_fuzz_points = 100
cube_shape = Geometry.build_cube(d)

#draw a pair of cubes for each axis
for face, color in zip(cube_shape.faces,tess_face_colors):
    face.color = color
    #face.color = GREEN
shapes = []
r = 4
for i in range(d):
    cube = cube_shape.copy()
    cube.update(pos = r*np.eye(d)[i])
    shapes.append(cube)
    cube = cube_shape.copy()
    cube.update(pos = -r*np.eye(d)[i])
    shapes.append(cube)
#shapes = shapes[:2] + shapes[-4:] 
#shapes = shapes[:2] + shapes[-2:]
#shapes.pop(2) #removes top cube
#cube_shape.transparent = True
shapes = [shapes[0] ]
for i,shape in enumerate(shapes):
    shape.name = 'shape ' + str(i)
    
floor_lines = Geometry.build_floor(d,1,16,-5)

## other shapes

In [20]:
d = 3
sphere_shape = Geometry.build_sphere_3d(r=1,phi_pts=10,theta_pts=5)
shapes = [sphere_shape]
#shapes.append(sphere_shape)

In [4]:
d = 4
m = 5; n = 5
cylinder_shape = Geometry.build_duocylinder(rs=[1,1],axes=[[0,1],[2,3]],n_circ_pts=[m,n])
#cylinder_shape.transparent = True
for i,face in enumerate(cylinder_shape.faces[:m]):
    face.color = ORANGE
    #face.color = (np.cos(2*np.pi*i/m)**2,np.sin(2*np.pi*i/m)**2,1)
    #face.color = ((i+1)%2,(i+1)%2,i%2)
for i,face in enumerate(cylinder_shape.faces[m:]):
    face.color = CYAN
    #face.color = (1,np.cos(2*np.pi*i/m)**2,np.sin(2*np.pi*i/m)**2)
    #face.color = ((i+1)%2,i%2,(i+1)%2)
    #face.color = ((i+1)%2,(i+1)%2,i%2)
#shapes = [cylinder_shape]
shapes.append(cylinder_shape)

# comments

Would like draw.py to have clearer distinctions between the various spaces. The layers are
1. The d-dim local spaces that shape.verts_ref, ref_frame, and face.normal_ref etc live in
2. The d-dim world space that rotated, scaled and positioned shapes (and faces) and cameras live
3. The d-dim space rotated and translated wrt to a camera
4. The (d-1)-dim space that shapes are projected on to
5. The (d-1)-dim graphics backend (openGL)

It would be nice if we could organize code according to these layers.

Clipping happens in 2

Todo: 
- fix target shape clipping
- fix viewport clipping
- speed up /generalize floor drawing
- speed up clipping
- find a way to make the center obvious wrt objects in the view. does rotation help or hurt? how can one navigate with a rotating viewport?
- other points of reference at infinity
- a floor and ceiling
- implement cubinders, spherinders
- generalize face fuzz to all convex faces, invariant to edge order

# main loop

In [17]:


t = 0
#size = width, height = 1100, 700
size = width, height = 1600, 900

pygame.init()

if d == 3:
    draw.init(d,size,focal=6,n_fuzz_points = 100)
    draw.view_radius = 7
    draw_view_height = 7
    draw.view_boundary = 'none'
    camera = Camera(pos = Point([0,0,-10]))
    camera.target_shape = Geometry.build_3d_target()

    camera.look_at(vec.zero_vec(d))

if d == 4:
    draw.init(d,size,stereo=False,n_fuzz_points = 200)
    draw.view_boundary = 'none'
    if draw.stereo:
        draw.view_radius = 4
    else:
        draw.view_radius = 6
        draw.view_height = 4

    draw.face_scales = [0.9]

    camera = Camera(pos = vec.one_hot(d,-1)*(-5))
    camera.look_at(vec.zero_vec(d))

draw.clipping = True
quit = False
redraw = True
timer = time.time()
t0 = time.time()
frame = 0
try:
    while not quit:

        if redraw:
            draw.draw(camera,shapes+ [camera.target_shape])
            draw.clip_and_draw_lines(floor_lines,camera,(0.2,0.2,0.2),None,shapes)
            #test_points(camera,shapes[0].pos)
            #draw.draw_wireframe(camera,cylinder_shape,GRAY)
            #draw.draw_normals(camera,shapes[0],WHITE)
            pygame.display.flip()
            redraw = False
            
        #this is here so that we draw on the first frame
        redraw, quit = input_update(camera, shapes)
        #camera.update_rot_matrix(0, 2, camera.ang_speed/2)
        #sphere_shape.rotate(0,2,0.01)
        #cylinder_shape.rotate(1,3,0.02)
        redraw = True #redraw every loop. useful for assessing framerate


        delta_t = (time.time() - timer)
        if delta_t > 2:
        #if False:
            #print('redraw:',redraw)
            try:
                #fps = 30/(time.time() - timer)
                fps = frame/delta_t
            except:
                fps = 0
            #if fps < 60:
            print(fps,'fps')
            timer = time.time()
            frame = 0
        frame = frame+1
except:
    pygame.quit()
    print('game terminated')
    raise
pygame.quit()
print('game terminated')

init GL
46.203417370895075 fps
48.02243240404012 fps
47.98101604077347 fps
47.479771053582965 fps
48.02375883460839 fps
48.00001321380035 fps
47.52838241767114 fps
transparency True
46.486256847741316 fps
46.999904751970675 fps
24.752344013996275 fps
23.999711040161245 fps
23.998056997238695 fps
transparency False
24.484889236500578 fps
47.49943942493164 fps
48.05497376478702 fps
47.94205992180261 fps
47.504130874172915 fps
47.52299727823377 fps
48.065419939126606 fps
47.99790010102545 fps
48.00006984445569 fps
48.00072110659659 fps
47.49654616199031 fps
48.01272448314753 fps
47.932837180112 fps
47.998733396624175 fps
48.03097612744809 fps
47.965126597033105 fps
game terminated


In [188]:
pygame.quit()

# workspace